# Enriching Companies House data

![title](Images/ch_screenshot.png)

This notebook gives an overview on how business data from Companies House is imported, formatting and then enriched by exploiting various APIs and websites. Examples of applications include:
-  Retrieving official company websites using Google Places API
-  Scraping websites to get keywords to classify the industry of businesses.
-  Obtaining social media accounts and handles for companies and then use these to get a proxy for their web presence (number of followers, likes etc.)

## Importing data

The Free Company Data Product is a downloadable data snapshot containing basic company data of live companies on the Companies House register, and is the principal dataset for this project. This is updated monthly and needs to be downloaded before importing as a pandas dataframe. First, we need to import some modules...

### Modules

-  Pandas: provide easy-to-use data structures in Python
-  Numpy: provides fast and efficient multidimensional arrays, in addition to linear algebra and mathematical operations.
-  Matplotlib: provides plot to visualise data

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

import matplotlib.pyplot as plt
# Increase figure and font sizes for easier viewing
plt.rcParams['figure.figsizeb'] = (8, 6)
plt.rcParams['font.size'] = 14

### Loading and formatting Companies House dataset

The latest version of the Free Company Data Product can be downloaded here. http://download.companieshouse.gov.uk/en_output.html. The zip file that is downloaded is approximately 300MB, and the raw CSV file around 2GB. Once downloaded, ensure that the data is saved in the root folder of this notebook (or amend directory as required).

In [ ]:
# to-do: investigate warning on mixed data types
ch_raw = pd.read_csv('/Users/dataexploitationmac1/Desktop/Faisal/Datasets/BasicCompanyDataAsOneFile-2018-02-01.csv')
# ch_raw = pd.read_csv('/Users/fasamin/Desktop/DS/Datasets/BasicCompanyDataAsOneFile-2018-02-01.csv')

In [ ]:
# preview the data
ch_raw.head(10) # first 10 rows 

In [ ]:
# fields available
ch_raw.columns

In [ ]:
# remove unnecessary columns for this project
# why is copy() used? See explanation at link below:
# https://stackoverflow.com/questions/27673231/why-should-i-make-a-copy-of-a-data-frame-in-pandas
ch = ch_raw.iloc[:,[0,1,4,5,6,7,8,9,10,11,12,18,19,21,26,27,28,29]].copy()

# rename columns
ch.columns = ['name','crn','address1','address2','postTown','county','country', \
            'postcode','category','status','origin','accounts_lastMadeUpDate','accountCategory',\
            'returns_lastMadeUpDate','sic1','sic2','sic3','sic4']

In [ ]:
# format missing values
ch.sic1.replace('None Supplied', np.NaN, inplace=True)
ch = ch.dropna(subset=['name']) # delete rows with null business names (usually only a few values)

In [ ]:
# Produce a range of key stats 
# Key stats
print('---------')
print('Number of businesses: %s' %len(ch))
print('Missing SIC codes: %s' %ch.sic1.isnull().sum())
sic_comp = (1.0 - (float(ch.sic1.isnull().sum())/len(ch)))*100
print('SIC code completion: %.2f' %sic_comp + '%')
post_comp = (1.0 - (float(ch.postcode.isnull().sum())/len(ch)))*100
print('Postcode completion: %.2f' %post_comp + '%')
print('---------')
print('Category breakdown (top 5)')
print('')
print(ch.category.value_counts().head())
print('---------')
print('Account category (top 5)')
print('')
print(ch.accountCategory.value_counts().head())
print('---------')
print('Geographical breakdown (top 5)')
print('')
print(ch.origin.value_counts().head())
print('---------')
print('SIC code breakdown (top 5)')
print('')
print(ch.sic1.value_counts().head())

### Data exploration

Some pandas commands to explore the dataset, including setting up a function to find companies.

In [ ]:
ch.dtypes # types of each column - all objects

In [ ]:
def find_company(name):
    '''
    Searches companies house dataset for company name which include the given input which must be a string.
    '''
    name = name.lower()
    n = ch.name.str.lower().str.contains(name)
    x = input(str(n.sum()) + ' companies found. See list of companies? Y or N? ')
    if x.lower() == 'y':
        return ch[n]
    else:
        return True

In [ ]:
find_company('Burberry') # testing function on a few cases 

In [ ]:
find_company('Dyson')

Exploring SIC codes...

In [ ]:
ch.sic1.describe() # counts occurences and unique values

Sorting by the top 20 SIC codes shows that some of these are not very descriptive. Top of the list is 'Other business support service activities n.e.c'. Third is 'Dormant Company' and this is followed by 'Other service activities n.e.c'.

In [ ]:
ch.sic1.value_counts().head(20) # sort by top 20 sic codes

In [ ]:
ch.sic1.value_counts().head(30).plot() # shows skew of top categories
plt.show()

Checking if company reference numbers are unique

In [ ]:
ch.crn.describe() # all crns are unique

In [ ]:
ch.crn.isnull().sum() # 0

Exploring the address data

In [ ]:
ch.head() # reminder of the address fields

In [ ]:
ch.address1.describe() # 1.6 million unique address

In [ ]:
ch.address1.isnull().sum() # 27K null addresses

In [ ]:
ch.postTown.isnull().sum() # 93K missing town names

In [ ]:
ch.postcode.isnull().sum() # 52K missing post codes

### Export formatted dataset

In [ ]:
# Export dataset, named after MMYY of ch data
ch.to_csv('ch_2018-02.csv',index=False)

Optional: remove non-UK companies 

In [ ]:
ch_uk = ch[ch['origin'].isin(['United Kingdom','Great Britain','UNITED KINGDOM','GREAT BRITAIN','ENGLAND & WALES','UK'])]
ch_uk.reset_index(inplace=True)
ch_uk.to_csv('ch_2018-02_uk.csv',index=False)

## Scraping data from Google Search Results

This section goes through the process of running google searches of business names in Companies House, scraping text from the results, and then returning a wordcloud of text from the first page of results.

The code below builds up the code for functions that run searches and produce
worldclouds as follows:

cloud(keyWords(search('Company Name')))

- search(string): returns a list of URLs from Google for the given term
- keyWords(list): screen-scrapes all visible text from the given list of URLs, and cleans
- cloud(string): after removing a given list of stopwords, produces a wordcloud

### Importing further modules and data

In [ ]:
import webbrowser # to open web links
import nltk # natural language toolkit
from nltk.corpus import stopwords # Import the stop word list, may require download
# WordCloud modules
from wordcloud import WordCloud, STOPWORDS

import re # regular expressions 
from time import sleep # to pause web-scraper
import requests # allows you to send HTTP requests via Python
from bs4 import BeautifulSoup # beautiful soup for parsing of HTML

Read in formatted CH dataset if starting a new session. We'll refer to this at the end of the section after we've built up our tools to scrape and clean website text.

In [ ]:
ch = pd.read_csv('ch_2018-02.csv')

### Returning links for a Google Search term

First, we need to build some functionality to scrape the search results returned by Google.

In [ ]:
# Set business search term as an example
biz = 'DYSON LIMITED'

In [ ]:
# Read HTML
html = requests.get('https://www.google.co.uk/search?q='+ biz)
# Parse HTML into a BeautifulSoup object
soup = BeautifulSoup(html.content, 'html5lib')

In [ ]:
# Get all links and put into list
list_of_links = []
for link in soup.find_all('a'):
    list_of_links.append(link.get('href'))

In [ ]:
print(list_of_links) # needs cleaning up 

In [ ]:
# Cleaning up results
links = DataFrame({'urls':list_of_links}) #turn list into DF
links = links[links.urls.str.contains('/url?')] #Only search results

In [ ]:
# remove cached sites
links = links[links.urls.str.contains('webcache.googleusercontent') == False]

In [ ]:
# remove opening url?q= string
links = links.urls.str.replace('/url\?q=',"")
# after this, you don't need to call list anymore on the column

In [ ]:
# remove suffixed &sa bit by splitting and drop index
links = links.str.split('&sa',1).reset_index().drop('index',1)
# this is now a dataframe

In [ ]:
# use iterrows to grab first entry in each list which should be the working url
links_cleaned = []
for row in links.iterrows():
    links_cleaned.append(row[1][0][0])

In [ ]:
# convert to dataframe
links_cleaned = DataFrame(links_cleaned)

We can now bring this together in one function. Note that we're screen-scraping from Google Search results so we'll need to be careful to not overload Google with search requests in quick succession (and potentially get our IP blocked).

In [ ]:
def search_google(business_name):
    '''
    Takes in a business name and returns the links returned in the first page of Google Search results
    '''
    # Read HTML
    html = requests.get('https://www.google.co.uk/search?q='+business_name)
    # Parse HTML into a BeautifulSoup object
    soup = BeautifulSoup(html.content, 'html5lib')

    # Get all links and put into list
    list_of_links = []
    for link in soup.find_all('a'):
        list_of_links.append(link.get('href'))

    # Cleaning up results
    links = DataFrame({'urls':list_of_links}) #turn list into DF
    links = links[links.urls.str.contains('/url?')] #Only search results
    
    # remove cached sites
    links = links[links.urls.str.contains('webcache.googleusercontent') == False]

    # remove opening url?q= string
    links = links.urls.str.replace('/url\?q=',"")
    # after this, you don't need to call list anymore on the column

    # remove suffixed &sa bit by splitting and drop index
    links = links.str.split('&sa',1).reset_index().drop('index',1)
    # this is now a dataframe

    # use iterrows to grab first entry in each list which should be the working url
    links_cleaned = []
    for row in links.iterrows():
        links_cleaned.append(row[1][0][0])
        
    # convert to dataframe
    links_cleaned = DataFrame(links_cleaned)

    return links_cleaned[0]

In [ ]:
burberry_links = search_google('burberry limited')

In [ ]:
# use webbrowser library to open all links in browser (if needed)
for link in burberry_links:
    webbrowser.open(link)

### Extract key text from company websites 

Now that we've got the functionality to return links from Google search results, we want to navigate to each link, scrape and format the text to find words with explanatory value after removing stopwords and other standard website text. Let's use the Dyson website as an example. 

In [ ]:
dyson_links = search_google('Dyson Limited')

In [ ]:
# Inspecting the first search result
# Read HTML
html = requests.get(dyson_links[0])
# Parse HTML into a BeautifulSoup object
soup = BeautifulSoup(html.content, 'html5lib')

In [ ]:
# Extracting text from key html sections
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]

In [ ]:
# Get all visible text
text = soup.getText().encode('ascii','ignore')

We want to clean up this text by removing HTML tags and new line indicators. 

In [ ]:
text_r = str(text).replace('\\n','').replace('\\t','').replace('\\r','')
print(text_r)

In [ ]:
def stripsymbols(text):
    '''
    Use regular expressions to do a find-and-replace of HTML text.
    Function found online
    '''
    text = str(text)
    x = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
    x = re.sub('/(^|\b)@\S*($|\b)/'," ",x)
    x = re.sub('/(^|\b)#\S*($|\b)/'," ",x)
    x = re.sub("[^a-zA-Z]"," ",x)
    x = re.sub(r"(?:\@|https?\://)\S+", " ",x)
    return x

In [ ]:
text_stripped = stripsymbols(text_r)
print(text_stripped)

Getting better! The final step is remove stop words by invoking the Natural Language Toolkit Library we imported earlier. Let's see what kind of stop words are classified in the library. 

In [ ]:
print(stopwords.words('English'))

We can create a function to remove stopwords from website text.

In [ ]:
def removeStopWords(text):
    '''
    Remove stopwords from given text string
    '''
    words = [w for w in text if not w in stopwords.words("english")]
    return words

In [ ]:
# Prepare text for function by (a) changing to lowercase (2) remove whitespace from beginning and end 
# (3) splitting the text to create a list of individual wordsvisible_text_stripped.lower().strip().split() 
text_stripped_split = text_stripped.lower().strip().split()
text_stripped_split

Finally, running this through the removeStopWords function gives us something in a much better shape than the original text. 

In [ ]:
text_cleaned = removeStopWords(text_stripped_split)
text_cleaned

Wrapping this all up into a function...

In [ ]:
def cleantext(text):
    text_r = str(text).replace('\\n','').replace('\\t','').replace('\\r','')
    text_stripped = stripsymbols(text_r)
    text_stripped_split = text_stripped.lower().strip().split()
    text_cleaned = removeStopWords(text_stripped_split)
    return ' '.join(text_cleaned) # returns a joined list of the remaining words 

In [ ]:
dyson_cleaned = cleantext(text) # example of a function
dyson_cleaned

### Using the Natural Language Toolkit to tokenise and tag words

Another feature of the Natural Language Toolkit we can exploit is the ability to tag words and categorise them into their 'parts of speech' (i.e. nouns, verbs, adverbs). Perhaps this can be used to tag words from the scraped text, and retrieve nouns under the assumption that they provide the greatest explanatory power.

Let's test this with the scraped text from the previous section.

In [ ]:
dyson_cleaned

In [ ]:
tokens = nltk.word_tokenize(dyson_cleaned) # tokenize (split) the string
print(tokens)

The tags bellow are given according to the Penn Treebank Project here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

The results are quite interesting. Words like 'vacuums', 'cylinders', 'robot' and 'dryers' are correctly given as nouns. However, some words are double-tagged. For instance, 'dyson' is given as a verb as well as a noun. 'Airblade' is given as a past-tense verb. 

From this example alone, filtering on nouns would yield the best results.

In [ ]:
tagged = nltk.pos_tag(tokens) # categorise words according to their parts of speech
print(tagged)

In [ ]:
# Keep nouns only
tagged_nouns = " ".join([word[0] for word in tagged if word[1] in ['NNS','NN']])
tagged_nouns

In [ ]:
# Difference in list sizes - 1,352 removed words (out of 3,767)
len(dyson_cleaned) - len(tagged_nouns)

### Master function and wordclouds

Let's collect everything we've built in Section 2 and build our final master functions.

In [ ]:
def key_words(list_of_urls):
    '''
    From the given list of urls, this function scrapes, cleans (removing symbols and non-nouns)
    and gathers all text into a single string
    '''
    
    words_cleaned = '' # Set up empty string
    
    for website in list_of_urls:
        # try / except for troublesome websites
        try:
            html = requests.get(website)
            soup = BeautifulSoup(html.content, 'html5lib')
            [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
            visibleText = soup.getText().encode('ascii','ignore')
            words_cleaned += cleantext(visibleText) # Calling the function we built earlier 
        except:
            continue
            
    # Tokenize and tag words
    tokens = nltk.word_tokenize(words_cleaned)
    tagged = nltk.pos_tag(tokens)

    # Keep nouns only
    keywords = " ".join([word[0] for word in tagged if word[1] in ['NNS','NN']])
    return keywords

In [ ]:
key_words(search_google('Dyson Limited')) # testing on an example - takes a few seconds to run

This is best visualised in a word cloud...

In [ ]:
def cloud(words):
    '''
    Function that takes in a string and produces a word cloud
    '''
    wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='black',
                          width=1800,
                          height=1400
                         ).generate(words)

    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    return wordcloud

In [ ]:
cloud(key_words(search_google('Royal Dutch Shell'))) # can take upto a minute to run

In [ ]:
x = cloud(key_words(search_google('carillion')))

There is a lot of scope to improve this. Certain terms from websites that commonly re-occur can be filtered out by updating the stopwords set as follows.

In [ ]:
# ways to improve this: list of common names, weights and quantities, months and dates,

custom_words = ['sun','new','showbiz','tv','uk','john','lewis','partnership','offers','store',
'business','company','stores','shop','department','partner','street','london','partners','peter',
'jones','duration', 'views', 'minutes', 'month', 'version', 'system','tesco','september','privacy',
'policy','customer','service','home', 'company', 'london', 'price', 'offer', 'customer',
'service', 'home', 'year', 'london', 'day', 'march', 'business', 'shop','item','level','logo','menu',
'account','co','road','centre']


#STOPWORDS is a set, so need to use update method
STOPWORDS.update(custom_words)

## Connecting to Google Places API

We've relied on Google Search results to hopefully get websites that relate to a company term. However, the Google Places API can be linked to with a company's name and postcode and used to retrieve the company website and other details. 

Information from Google Places is displayed for certain businesses and places of interest as shown below.

![title](Images/google_places_screenshot.png)

### Modules and setup

We begin as usual by importing the required modules

In [ ]:
import pandas as pd
import numpy as np
import requests
from time import sleep # to pause requests when web-scraping

Google Places API requests can be called with the requests package. By default, you'll be able to send 1,000 requests per day but this can be increased free of charge, up to 150,000 requests per 24 hour period, by enabling billing on the Google API Console to verify your identity. Check quota here (after registering): http://code.google.com/apis/console

In [ ]:
YOUR_API_KEY = '' # enter API key which you'll receive from Google 

Check documentation here for info on the various Google Places APIs: https://developers.google.com/places/web-service/

Let's use the postcode for London Eye as an example: SE1 7PB. We'll need to input the longitude and latitude for this

In [ ]:
postcode = 'SE1 7PB'

In [ ]:
html = requests.get("http://api.postcodes.io/postcodes/" + postcode)
html.json()

In [ ]:
lat = html.json()['result']['latitude']
lon = html.json()['result']['longitude']
lat, lon

### Nearby search

A Nearby Search lets you search for places within a specified area. You can refine your search request by supplying keywords or specifying the type of place you are searching for. A nearby search request is an HTML URL of the following form:

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

Using the London Eye as an example:

In [ ]:
london_eye = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=' + \
                          str(lat) + ',' + str(lon)  + '&' + 'keyword=London+Eye' + '&' + \
                          'rankby=distance' + '&' + \
                          'key=' + str(YOUR_API_KEY))
london_eye.json()

Exploring the json() file...

In [ ]:
london_eye.json()['results'] # list of results


In [ ]:
london_eye.json()['results'][0]['name'] # name of place

In [ ]:
le_id = london_eye.json()['results'][0]['place_id'] # id of location 
le_id

Now that we've got the id, we can get the website from the text search API.

Before leaving this section, let's build a helper function that returns the location id from a given set of parameters. We'll need to call upon this function later.

In [ ]:
def nearby_search(lat,lon,keyword,api_key):
    '''
    input: latitude and longitude (floats), api_key for Google Places (string)
    output: location_id of place closest to co-ordinates and that matches the keyword
    if the company is not found, then a message will be given
    '''
    
    # feed parameters to html 
    html = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=' + \
                        str(lat) + ',' + str(lon)  + '&' + 'keyword=' + keyword + '&' + \
                        'rankby=distance' + '&' + 'key=' + str(api_key))
        
    # Error message if location not found
    try: 
        place_id = html.json()['results'][0]['place_id']
    
    except:
        return 'Location not found'
    
    # return entire json for testing
    # return html.json()
    
    return place_id

In [ ]:
# testing the function
nearby_search(51.5028202620979,-0.119252376858172,'London Eye',YOUR_API_KEY) # correct

In [ ]:
# Returns a KFC about 1 mile from the London Eye. Since we can't restrict radius with the rankby parameter, 
# depending on how strict the business name matching is, we run the risk of mismatches
nearby_search(51.5028202620979,-0.119252376858172,'KFC',YOUR_API_KEY)

In [ ]:
nearby_search(51.5028202620979,-0.119252376858172,'London',YOUR_API_KEY) # returns 'London' - the entire city?

In [ ]:
nearby_search(51.5028202620979,-0.119252376858172,'Eye',YOUR_API_KEY) # Specsavers in the Strand

In [ ]:
nearby_search(51.5028202620979,-0.119252376858172,'Wheel',YOUR_API_KEY) # Kwik-fit in Elephant & Castle

In [ ]:
nearby_search(51.5028202620979,-0.119252376858172,'Ferris Wheel',YOUR_API_KEY) # Correct - London Eye

In [ ]:
nearby_search(50.547036619931,-3.49639521511686,'R.B.W. DEVELOPMENTS LTD.',YOUR_API_KEY) # Testing sample

In [ ]:
place_search(nearby_search(54.288992488345,-1.59221709336977,'North Yorkshire Business Systems Ltd',YOUR_API_KEY))

In [ ]:
nearby_search(54.2500186970894, -1.05822691177972,'1 VOYAGE LIMITED', YOUR_API_KEY)

### Place search

Link here: https://developers.google.com/places/web-service/details

Once you have a place_id from a Place Search, you can request more details about a particular establishment or point of interest by initiating a Place Details request. A Place Details request returns more comprehensive information about the indicated place such as its complete address, phone number, user rating and reviews. A Place Details request is an HTTP URL of the following form:

In [ ]:
https://maps.googleapis.com/maps/api/place/details/output?parameters

Calling this on the London Eye place id..

In [ ]:
london_eye_website = requests.get('https://maps.googleapis.com/maps/api/place/details/json?' + \
                    'placeid=' + le_id + '&' + 'key=' + str(YOUR_API_KEY))

In [ ]:
london_eye_website.json()['result'] # list of results

In [ ]:
london_eye_website.json()['result']['formatted_address']

In [ ]:
london_eye_website.json()['result']['international_phone_number']

In [ ]:
london_eye_website.json()['result']['opening_hours'] # dictionary of opening hours

In [ ]:
london_eye_website.json()['result']['reviews'] # reviews

In [ ]:
london_eye_website.json()['result']['reviews'] # example of a review - NLP potential

In [ ]:
london_eye_website.json()['result']['website'] # place website

As before, let's build a helper function for convenience in the following section

In [ ]:
def place_search(place_id):
    '''
    input: place_id from Nearby Search API (string)
    output: company website (if found)
    If a website is not found, then an error message will be given
    '''
    
    # exception in case place_id doesn't yield results in Place Search
    try:
        html = requests.get('https://maps.googleapis.com/maps/api/place/details/json?' + \
                    'placeid=' + place_id + '&' + 'key=' + str(YOUR_API_KEY))
    except:
        return 'No results in Place Search'
    
    # exception if website is not found
    try:
        website = html.json()['result']['website']
    except:
        return 'Website not found'
    
    return website

In [ ]:
# Testing function with London Eye ID
place_search('ChIJc2nSALkEdkgRkuoJJBfzkUI')

In [ ]:
# Testing function with other IDs
place_search('ChIJmbKqQlgDdkgRfkx7-tWdNU4') # KFC in Lambeth - not branch-specific

In [ ]:
place_search('ChIJdd4hrwug2EcRmSrV3Vo6llI') # for London - website not avaiable

In [ ]:
place_search('ChIJDYZYNsoEdkgRHCQ30Uysy_M') # Specsavers - branch-specific

In [ ]:
place_search('ChIJWwBL554EdkgRluPLRtvCKek') # Kwik-fit - branch specific

In [ ]:
test = requests.get('https://maps.googleapis.com/maps/api/place/details/json?' + \
                    'placeid=' + 'ChIJadts_DgyeUgRSlzPEeIPLb8' + '&' + 'key=' + str(YOUR_API_KEY))

In [ ]:
test.json()

### Function to return website from given companies house details

Let's bring some of the pieces in the previous sections together to build a function that returns the company website (if found) from details found in the publicly-available Companies House dataset.

In Section 1.4, we exported a cleaned-up dataset of Companies House to a local directly. Let's import this now.

In [ ]:
ch = pd.read_csv('ch_2018-02.csv')

In [ ]:
ch.shape

We'll be running the business name and the postcode through the Google Places API. The business name should be given but we'll need to exclude those records where the postcode is not given.

In [ ]:
ch_w_pc = ch[ch.postcode.notnull()].copy()
ch_w_pc.shape

Ok, so we've got our dataset. Let's now put our function together.

In [ ]:
def return_website(company_name,postcode):
    '''
    Input: company name and postcode (both strings)
    Output: Company website
    '''
    
    # Step 1: Convert postcodes to latitude and longitude using postcodes.io
    # Some companies may give invalid postcodes in Companies House so build exception
    
    try: 
        html = requests.get("http://api.postcodes.io/postcodes/" + postcode)
        lat = html.json()['result']['latitude']
        lon = html.json()['result']['longitude']
    except:
        return 'Postcode not found'
    
    # Step 2: Get place_id by calling upon a Nearby Search (see Section 3.2)
    place_id = nearby_search(lat,lon,company_name,YOUR_API_KEY)
    
    # Step 3: Return website by feeding place_id into Place Search
    website = place_search(place_id)
    
    return website    

### Testing function on Companies House

Now for the fun stuff: let's test this function on a sample of companies in Companies House and see how many companies we get websites for.

In [ ]:
# Random sample of a 100 companies from those entries with a postcode, only return name and postcode
ch_random_100 = ch_w_pc.sample(100)[['name','postcode']].copy()

In [ ]:
ch_random_100.loc['website'] = np.NAN # set up blank column
ch_random_100

In [ ]:
# Retrieve websites for each company with a for loop
i = 1
for index, row in ch_random_100.iterrows():
    ch_random_100.loc[index,'website'] = return_website(row[0],row[1])
    print(str(i) + ' - ' + str(row[0]) + ' - ' + str(row[1]) + ' - ' + str(row[2]))
    i += 1
    sleep(0.5)

In [ ]:
ch_random_100['website'].value_counts().head() # 17 websites out of 100 

In [ ]:
ch_random_100.to_csv('sample_of_100_websites.csv') # save results

Let's see if we can improve the match-rate by removing 'LTD. and 'LIMITED' from the end of the name

In [ ]:
ch_random_100.loc[:,'website_2'] = np.NAN # set up blank column
ch_random_100

In [ ]:
ch_random_100 = pd.read_csv('sample_of_100_websites.csv',index_col=0)

In [ ]:
ch_random_100 = ch_random_100.head(100).copy()

In [ ]:
ch_random_100

In [ ]:
def standardise(company_name):
    stopwords = ['limited','ltd','ltd.','lp']
    querywords = str(company_name).split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result

In [ ]:
ch_random_100['name_stripped'] = ch_random_100['name'].apply(standardise)

In [ ]:
ch_random_100

In [ ]:
# Retrieve websites for each company with a for loop
i = 1
for index, row in ch_random_100.iterrows():
    ch_random_100.loc[index,'website_2'] = return_website(row[4],row[1])
    print(str(i) + ' - ' + str(row[4]) + ' - ' + str(row[1]) + ' - ' + str(row[3]))
    i += 1
    sleep(0.5)

In [ ]:
ch_random_100['website_2'].value_counts().head() # 18 websites out of 100, lose some cases, gain others

In [ ]:
ch_random_100.to_csv('sample_of_100_websites_2.csv') # save results

### Testing on NYER LEP

Import postcode lookups for the North Yorkshire and East Riding LEP from the National Postcode Lookup Table

In [ ]:
nyer_postcodes = pd.read_csv('nyer_postcode.csv')

In [ ]:
nyer_postcodes.head() # three postcodes to lookup for 

Let's do an inner join from the CH data to the NYER postcodes on the POSTCODE_PCDS column.

In [ ]:
ch_nyer = pd.merge(ch_w_pc, nyer_postcodes, how='inner', left_on='postcode', right_on='POSTCODE_PCDS')

In [ ]:
ch_nyer.shape # 12,486 companies. Similiar to what we're seeing in CSAT.

In [ ]:
ch_nyer.loc[:,'website'] = np.NAN # set up blank column
ch_nyer

In [ ]:
# Retrieve websites for each company with a for loop
for index, row in ch_nyer.iterrows():
    if pd.notnull(row[28]) == True:
        continue
    ch_nyer.loc[index,'website'] = return_website(row[0],row[7]) 
    print(str(index) + ' - ' + str(row[0]) + ' - ' + str(row[7]) + ' - ' + str(ch_nyer.loc[index,'website']))
    sleep(0.2)

In [ ]:
ch_nyer['website'].value_counts() # 3,176 websites found for 12,486 (25%), some of these are duplicates

In [ ]:
ch_nyer.to_csv('nyer_with_websites.csv')

## Scraping social media accounts 

Now that we have the websites from Google Places, we can scrape social media accounts and other interesting sites (i.e. trip advisor) for these and then subsequently scrape activity, and follower information from those.

Let's import our NYER dataset from the previous section.

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

In [38]:
nyer = pd.read_csv('nyer_with_websites.csv', index_col=0) 

In [ ]:
nyer_websites = nyer[nyer['website'] != 'Website not found'] # select those with websites

In [ ]:
nyer_websites.head(10)

Choosing 'AUTO 66 RACING LIMITED', with crn 00980520 for our test as this has social media handles.

In [ ]:
auto = nyer_websites[nyer_websites['crn'] == '00980520']

In [ ]:
auto

In [ ]:
print(auto.website.values) # website

Python package exists to extract social media: https://pypi.python.org/pypi/extract-social-media/0.4.0. Can also used regex: https://github.com/lorey/social-media-profiles-regexs. We'll test the package for now.

In [ ]:
# Code copied and pasted from above link
import requests
from html_to_etree import parse_html_bytes
from extract_social_media import find_links_tree # install first with setup.py from link above

In [ ]:
res = requests.get('http://www.oliversmountracing.com/')
tree = parse_html_bytes(res.content, res.headers.get('content-type'))

In [ ]:
set(find_links_tree(tree))

Wrapping this up in a function

In [37]:
def get_social_media(url):
    '''
    Retrieve social media accounts from a company website
    Input: string of URL
    Output: dictionary of links
    '''
    try:
        res = requests.get(url)
        tree = parse_html_bytes(res.content, res.headers.get('content-type'))
        return set(find_links_tree(tree))
    except:
        return 'Not found'

In [ ]:
get_social_media('http://www.oliversmountracing.com/')

In [ ]:
nyer_websites.website

Running this on dataset (took about an hour to run)

In [39]:
nyer['social_media'] = nyer.website.apply(get_social_media)

### Getting twitter followers

In [36]:
nyer_websites.head()

,name,crn,address1,address2,postTown,county,country,postcode,category,status,...,POSTCODE2,POSTCODE_PCDS,LAT,LONG,LEP1,LEP2,COUNTY_CODE,COUNTY_DIST_CODE,WARD_CODE,website
0,"""STREONSHALH"" WHITBY LIMITED",01636551,"""STREONSHALH""",KHYBER PASS,WHITBY,NORTH YORKSHIRE,NaN,YO21 3DQ,"PRI/LTD BY GUAR/NSC (Private, limited by guara...",Active,...,YO21 3DQ,YO21 3DQ,54.488915,-0.615535,E37000039,NaN,E10000023,E07000168,E05006340,http://www.streonshalh.co.uk/
4,WORLD WIDE SHOPPING MALL LIMITED,03307834,HILL VIEW HOUSE,"CORNBOROUGH ROAD, SHERIFF HUTTON",YORK,NORTH YORKSHIRE,NaN,YO60 6QJ,Private Limited Company,Active,...,YO60 6QJ,YO60 6QJ,54.091054,-1.009659,E37000039,NaN,E10000023,E07000167,E05006313,http://www.worldwideshoppingmall.co.uk/
6,1 VOYAGE LIMITED,04683525,6 FEVERSHAM ROAD,NaN,HELMSLEY,NORTH YORKSHIRE,NaN,YO62 5HN,Private Limited Company,Active,...,YO62 5HN,YO62 5HN,54.250017,-1.058227,E37000039,NaN,E10000023,E07000167,E05006302,http://www.voyagecare.com/
17,A & D MOTOR SERVICES (SCARBOROUGH) LIMITED,09104365,62/63 WESTBOROUGH,NaN,SCARBOROUGH,NaN,NaN,YO11 1TS,Private Limited Company,Active,...,YO11 1TS,YO11 1TS,54.278977,-0.408069,E37000039,NaN,E10000023,E07000168,E05006317,http://www.scarboroughdialaride.org/
23,ANDREW COWEN ESTATE AGENT LIMITED,03696005,62/63 WESTBOROUGH,SCARBOROUGH,NORTH YORKSHIRE,NaN,NaN,YO11 1TS,Private Limited Company,Active,...,YO11 1TS,YO11 1TS,54.278977,-0.408069,E37000039,NaN,E10000023,E07000168,E05006317,http://www.andrewcowen.com/


In [42]:
nyer.social_media.head(100)

0                                                    {}
1                                             Not found
2                                             Not found
3                                             Not found
4       {https://plus.google.com/110394507306641485115}
5                                             Not found
6     {https://twitter.com/voyagecare, https://www.l...
7                                             Not found
8                                             Not found
9                                             Not found
10                                            Not found
11                                            Not found
12                                            Not found
13                                            Not found
14                                            Not found
15                                            Not found
16                                            Not found
17              {http://www.twitter.com/tweetdia

In [51]:
nyer_sm = nyer[(nyer['social_media'] != 'Not found') & (nyer['social_media'] != {})] # select those with websites

In [52]:
nyer_sm.social_media

0                                                       {}
4          {https://plus.google.com/110394507306641485115}
6        {https://twitter.com/voyagecare, https://www.l...
17                 {http://www.twitter.com/tweetdialaride}
23                                                      {}
26       {https://twitter.com/auto66racing, https://www...
59                                                      {}
65                                                      {}
66                                                      {}
67                                                      {}
73       {https://plus.google.com/116251513247505766784...
76                                                      {}
92                                                      {}
93                                                      {}
95                                                      {}
99                                                      {}
100                                                     

In [55]:
nyer_sm.social_media.isnull().sum()

0

In [56]:
nyer.head()

,name,crn,address1,address2,postTown,county,country,postcode,category,status,...,POSTCODE_PCDS,LAT,LONG,LEP1,LEP2,COUNTY_CODE,COUNTY_DIST_CODE,WARD_CODE,website,social_media
0,"""STREONSHALH"" WHITBY LIMITED",01636551,"""STREONSHALH""",KHYBER PASS,WHITBY,NORTH YORKSHIRE,NaN,YO21 3DQ,"PRI/LTD BY GUAR/NSC (Private, limited by guara...",Active,...,YO21 3DQ,54.488915,-0.615535,E37000039,NaN,E10000023,E07000168,E05006340,http://www.streonshalh.co.uk/,{}
1,'OUT OF THIS WOLD' LTD,10553612,MILLHOLME FARM HOUSE,SPEETON STREET,FILEY,NaN,UNITED KINGDOM,YO14 9TG,Private Limited Company,Active,...,YO14 9TG,54.155893,-0.239788,E37000039,NaN,E10000023,E07000168,E05006327,Website not found,Not found
2,CLIFF TOP LTD,04623454,"MILLHOLME FARM HOUSE, SPEETON",FILEY,YORKSHIRE,NaN,NaN,YO14 9TG,Private Limited Company,Active,...,YO14 9TG,54.155893,-0.239788,E37000039,NaN,E10000023,E07000168,E05006327,Website not found,Not found
3,01722212 LIMITED,01722212,HILL VIEW HOUSE,"CORNBOROUGH ROAD, SHERIFF HUTTON",YORK,NORTH YORKSHIRE,NaN,YO60 6QJ,Private Limited Company,Active,...,YO60 6QJ,54.091054,-1.009659,E37000039,NaN,E10000023,E07000167,E05006313,Website not found,Not found
4,WORLD WIDE SHOPPING MALL LIMITED,03307834,HILL VIEW HOUSE,"CORNBOROUGH ROAD, SHERIFF HUTTON",YORK,NORTH YORKSHIRE,NaN,YO60 6QJ,Private Limited Company,Active,...,YO60 6QJ,54.091054,-1.009659,E37000039,NaN,E10000023,E07000167,E05006313,http://www.worldwideshoppingmall.co.uk/,{https://plus.google.com/110394507306641485115}


In [57]:
nyer.to_csv('nyer_with_websites_sm.csv')